In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
mpl.rc('font', family = 'Malgun Gothic') # 그래프 글꼴 설정 (window)

In [2]:
path = 'C:/Users/DMC CONET/Desktop/초미세먼지프로젝트/ty_data/result_output/'

# AirKorea

In [3]:
air = pd.read_csv(path + 'air_2015_2022.csv') # row: 69521 , column : 7 
air['날짜'] = pd.to_datetime(air['날짜'])
air

,날짜,PM10,PM2.5,오존,이산화질소,일산화탄소,아황산가스
0,2015-01-01 01:00:00,44,7,0.022,0.011,0.6,0.006
1,2015-01-01 02:00:00,57,10,0.022,0.010,0.6,0.006
2,2015-01-01 03:00:00,76,10,0.021,0.010,0.6,0.006
3,2015-01-01 04:00:00,86,11,0.022,0.009,0.6,0.006
4,2015-01-01 05:00:00,106,12,0.021,0.009,0.5,0.006
...,...,...,...,...,...,...,...
69516,2022-12-06 19:00:00,17,8,0.017,0.032,0.4,0.003
69517,2022-12-06 20:00:00,19,11,0.017,0.031,0.4,0.003
69518,2022-12-06 21:00:00,17,10,0.018,0.029,0.4,0.003
69519,2022-12-06 22:00:00,18,7,0.017,0.030,0.4,0.003


In [ ]:
weather.info()

# Weather

In [ ]:
weather = pd.read_csv(path + 'weather_2015_2022.csv')
weather['일시'] = pd.to_datetime(weather['일시'])

# weather['year'] = weather['일시'].apply(lambda x : x.year)
# weather['month'] = weather['일시'].apply(lambda x : x.month)
# weather['day'] = weather['일시'].apply(lambda x : x.day)
# weather['hour'] = weather['일시'].apply(lambda x: x.hour)
# weather

# AirKorea + Weather

In [ ]:
#결측치가 1건 발견되었음
air_weather['일시'].isnull().sum()

In [ ]:
# 확인해보니 2022 1월 1일 데이터가 없어 전값으로 채워도 무방하다고 판단.
air_weather = air.merge(weather, left_on='날짜', right_on='일시', how='left')#.sum()
cond = air_weather['일시'].isnull()
air_weather.loc[cond]

In [ ]:
air_weather.iloc[61360:61362]

# air_weather = air_weather.fillna(method='ffill') # 결측치를 이전값으로 채움

air_weather = air_weather.fillna(0) # 결측치를 이전값으로 채움
# air_weather['일시'].isnull().sum() # 결측치 없는것 확인

In [ ]:
air_weather.isnull().sum()

In [ ]:
# air_weather.to_csv('C:/Users/DMC CONET/Desktop/초미세먼지프로젝트/ty_data/result_output/air_weather.csv', encoding='utf-8-sig')

In [ ]:
df_input = air_weather[['날짜', 'PM10', 'PM2.5', '오존', '이산화질소', '일산화탄소', '아황산가스', 
            '기온(°C)', '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)',
            '증기압(hPa)', '이슬점온도(°C)', '현지기압(hPa)', '해면기압(hPa)', '지면온도(°C)',
       '5cm 지중온도(°C)', '10cm 지중온도(°C)', '20cm 지중온도(°C)', '30cm 지중온도(°C)']]

df_input['year'] = df_input['날짜'].apply(lambda x : x.year)
df_input['month'] = df_input['날짜'].apply(lambda x : x.month)
df_input['day'] = df_input['날짜'].apply(lambda x : x.day)
df_input['hour'] = df_input['날짜'].apply(lambda x: x.hour)
df_input

In [ ]:
import time

start = time.time()
df_input['6h_PM2.5'] = df_input['PM2.5'].mean() # 평균값으로 초기화

for i in range(6, len(df_input['6h_PM2.5'])):  # 6시간후라 range 6 부터 시작
    df_input['6h_PM2.5'].iloc[i-6] = df_input['PM2.5'][i]

print(f'코드 동작시간은 ? : {time.time() - start}')

In [ ]:
df_input['6h_PM2.5'] = df_input['6h_PM2.5'].astype(int)

In [ ]:
X = df_input[['PM10', 'PM2.5', '오존', '이산화질소', '일산화탄소', '아황산가스', 
            '기온(°C)', '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)',
            ]] # '증기압(hPa)', '이슬점온도(°C)', '현지기압(hPa)', '해면기압(hPa)', '지면온도(°C)',
       '5cm 지중온도(°C)', '10cm 지중온도(°C)', '20cm 지중온도(°C)', '30cm 지중온도(°C)',
             'year', 'month', 'day', 'hour'
y = df_input['6h_PM2.5']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12)

# XGBoost

pip install shap

In [ ]:
import numpy as np
import pandas as pd
import shap

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from matplotlib import pyplot as plt
import seaborn as sns # for correlation heatmap

from xgboost import XGBRegressor

In [ ]:
xgb = XGBRegressor(n_estimators=100)
xgb.fit(X_train, y_train)

In [ ]:
xgb.feature_importances_

In [ ]:
plt.figure(figsize=(20, 20))
plt.barh(X.columns.tolist(), xgb.feature_importances_)

나은 플롯을 얻으려 중요도 값을 기준으로 기능을 정렬

In [ ]:
X_fn = X.columns.tolist()

In [ ]:
sorted_idx = xgb.feature_importances_.argsort()
plt.figure(figsize=(20, 20))
plt.barh(X_fn, xgb.feature_importances_[sorted_idx])
plt.xlabel("Xgboost Feature Importance")

**Permutation Based Feature Importance**

In [ ]:
perm_importance = permutation_importance(xgb, X_test, y_test)

In [ ]:
sorted_idx = perm_importance.importances_mean.argsort()
plt.barh(X.columns.tolist(), perm_importance.importances_mean[sorted_idx])
plt.xlabel("Permutation Importance")

In [ ]:
def correlation_heatmap(train):
    correlations = train.corr()

    fig, ax = plt.subplots(figsize=(15,15))
    sns.heatmap(correlations, vmax=1.0, center=0, fmt='.2f', cmap="YlGnBu",
                square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .70}
                )
    plt.show();
    
correlation_heatmap(X_train[X.columns.tolist()])

**Feature Importance Computed with SHAP Values**

In [ ]:
explainer = shap.TreeExplainer(xgb)
shap_values = explainer.shap_values(X_test)

In [ ]:
shap.summary_plot(shap_values, X_test, plot_type="bar")

In [ ]:
shap.summary_plot(shap_values, X_test)

In [ ]:
shap.dependence_plot("hour", shap_values, X_test)

# RandomForestRegressor

In [ ]:
# 2. 지도학습에 필요한 라이브러리를 호출
from sklearn.model_selection import train_test_split
from sklearn.compose        import make_column_transformer
from sklearn.impute import KNNImputer
from imblearn.pipeline      import pipeline
from imblearn.pipeline      import make_pipeline
from sklearn.impute         import SimpleImputer, KNNImputer
from sklearn.preprocessing  import MinMaxScaler, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import ADASYN
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV    # 교차검증
from sklearn.metrics   import classification_report     # 평가
import pickle
from sklearn.metrics import r2_score, mean_squared_error

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor # Bagging Model 
from sklearn.tree import DecisionTreeClassifier


In [ ]:
# 4. 파이프라인을 구성한다.
numeric_pipe = make_pipeline(KNNImputer(), MinMaxScaler())
# category_pipe = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder())
numeric_list = X.describe().columns.tolist()
# category_list = X.describe(include='object').columns.tolist()

preprocessing_pipe = make_column_transformer((numeric_pipe, numeric_list)
#                                             , (category_pipe, category_list)
                                            )

model_pipe = make_pipeline(preprocessing_pipe, RandomForestRegressor())
# 5. 교차검증 및 하이퍼 파라미터 튜닝
hyperparameter_list = {'randomforestregressor__max_depth':range(5,11)
                        , 'randomforestregressor__min_samples_leaf':range(5,11)
                        , 'randomforestregressor__min_samples_split':range(5,11)}
grid_model = GridSearchCV(model_pipe, param_grid={}
                            , cv=5, n_jobs=-1, scoring='f1')# , error_score='raise'
grid_model.fit(X_train, y_train)

# 6. 평가
best_model = grid_model.best_estimator_

y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

print(f'y_train_pred : {y_train_pred}\n')
print(f'y_test_pred : {y_test_pred}\n')
# 평가
print(f'r2_score_train : {r2_score(y_train, y_train_pred)}\n')
print(f'r2_score_test : {r2_score(y_test, y_test_pred)}\n')

# encoder_list = best_model.named_steps['columntransformer'].transformers_[1][1].named_steps['onehotencoder'].get_feature_names().tolist()
# df_importance = pd.DataFrame()
# df_importance['Feature'] = pd.Series(numeric_list + encoder_list)


# df_importance['Importance']= best_model['randomforestregressor'].feature_importances_
# df_importance.sort_values(by='Importance', ascending=False)

print('explained_variance_score: {}'.format(explained_variance_score(y_train, y_train_pred)))
print('mean_squared_errors: {}'.format(mean_squared_error(y_train, y_train_pred)))
print('r2_score: {}'.format(r2_score(y_train, y_train_pred)))

print('explained_variance_score: {}'.format(explained_variance_score(y_test, y_test_pred)))
print('mean_squared_errors: {}'.format(mean_squared_error(y_test, y_test_pred)))
print('r2_score: {}'.format(r2_score(y_test, y_test_pred)))

In [ ]:
from sklearn.metrics import explained_variance_score
# 4. 파이프라인을 구성한다.
numeric_pipe = make_pipeline(KNNImputer(), MinMaxScaler())
# category_pipe = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder())
numeric_list = X.describe().columns.tolist()
# category_list = X.describe(include='object').columns.tolist()

preprocessing_pipe = make_column_transformer((numeric_pipe, numeric_list)
#                                             , (category_pipe, category_list)
                                            )

model_pipe = make_pipeline(preprocessing_pipe, RandomForestRegressor())

# 5. 교차검증 및 하이퍼 파라미터 튜닝
hyperparameter_list = {'randomforestregressor__max_depth':range(5,11)
                        , 'randomforestregressor__min_samples_leaf':range(5,11)
                        , 'randomforestregressor__min_samples_split':range(5,11)}
grid_model = GridSearchCV(model_pipe, param_grid=hyperparameter_list
                            , cv=5, n_jobs=-1, scoring='f1')# , error_score='raise'
grid_model.fit(X_train, y_train)

# 6. 평가
best_model = grid_model.best_estimator_

y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

print(f'y_train_pred : {y_train_pred}\n')
print(f'y_test_pred : {y_test_pred}\n')
# 평가
print(f'r2_score_train : {r2_score(y_train, y_train_pred)}\n')
print(f'r2_score_test : {r2_score(y_test, y_test_pred)}\n')

# encoder_list = best_model.named_steps['columntransformer'].transformers_[1][1].named_steps['onehotencoder'].get_feature_names().tolist()
# df_importance = pd.DataFrame()
# df_importance['Feature'] = pd.Series(numeric_list + encoder_list)


# df_importance['Importance']= best_model['randomforestregressor'].feature_importances_
# df_importance.sort_values(by='Importance', ascending=False)

print('explained_variance_score: {}'.format(explained_variance_score(y_train, y_train_pred)))
print('mean_squared_errors: {}'.format(mean_squared_error(y_train, y_train_pred)))
print('r2_score: {}'.format(r2_score(y_train, y_train_pred)))

print('explained_variance_score: {}'.format(explained_variance_score(y_test, y_test_pred)))
print('mean_squared_errors: {}'.format(mean_squared_error(y_test, y_test_pred)))
print('r2_score: {}'.format(r2_score(y_test, y_test_pred)))

# stats model - 통계적 접근

In [ ]:
# 실습용 데이터 패키지 
from sklearn.datasets import load_boston 

# 데이터 전처리 패키지 

import numpy as np 
import pandas as pd # 기계학습 모델 구축 및 평가 패키지 

import scipy as sp 
import scipy.stats as stats 
import statsmodels.api as sm 

from statsmodels.formula.api import ols 

from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error 



# CHOTE A 3X 

import seaborn as sns 
import matplotlib.pyplot as pit 
import matplotlib.font_manager as fm 
# 한글 폰트 설정 

plt.rc('font', family='Malgun Gothic')

In [ ]:
X_train = sm.add_constant(X_train)
model = sm.OLS(y_train, X_train, axis=1)
model_trained = model.fit()

In [ ]:
model_trained.resid

In [ ]:
model_residuals = model_trained.resid

plt.rcParams['axes.unicode_minus'] = False # 음수 폰트깨짐 방지
fig, ax = plt.subplots(1,1)
fig.set_figheight(12)
fig.set_figwidth(12)

sm.ProbPlot(model_residuals).qqplot(line='s', color='#1f77b4', ax=ax)
ax.title.set_text('QQplot')

**확률오차의 등분산성 확인**

In [ ]:
model_fitted_y = model_trained.fittedvalues

fig, ax = plt.subplots(1, 1)
fig.set_figheight(8)
fig.set_figwidth(12)

sns.residplot(model_fitted_y, y_train, data=X, lowess=True, scatter_kws={'alpha':0.5},
            line_kws={'color':'red'}, ax=ax)
ax.title.set_text('Residuals vs Fitted')
ax.set(xlabel='Fitted values', ylabel='Residuals')

**통계적 접근해보기**

- R_squard(결정계수) : 모형의 성능
- coef(회귀계수) : X가 한단위 증가할 때 Y의 변화량
- P >[t](p_value): 0.05(유의수준) 이하일 때 변수가 유의미

In [ ]:
print(model_trained.summary())

**p_value 0.05이상 : '10cm 지중온도(°C)', '5cm 지중온도(°C)', '일산화탄소', '이슬점온도(°C)'**

In [ ]:
model = sm.OLS(y_train, X_train.drop(columns=['10cm 지중온도(°C)', '5cm 지중온도(°C)', '일산화탄소', '이슬점온도(°C)']))
model_trained = model.fit()
print(model_trained.summary())

In [ ]:
model = sm.OLS(y_train, X_train.drop(columns=['10cm 지중온도(°C)', '5cm 지중온도(°C)', '일산화탄소', '이슬점온도(°C)', '현지기압(hPa)', '증기압(hPa)', '해면기압(hPa)', '지면온도(°C)']))
model_trained = model.fit()
print(model_trained.summary())

In [ ]:
# 성능에는 크게 변화가없지만, 개선이 직접적으로 되지않더라고 통계량값들의 해석을 할때 훨씬더 유의미한 해석이 이루어 질 수 있다.
# 스스로 개선 

In [ ]:
y_train_pred = model_trained.fittedvalues

plt.figure(figsize=(15, 15))
plt.title('실제값 vs 모델 출력 값')
plt.scatter(y_train, y_train_pred)
plt.plot([-5, 100], [-5, 100], ls='--', c='red')
plt.xlabel('실제값', size=16)
plt.ylabel('모델 출력 값', size=16)
plt.xlim(-5, 100)
plt.ylim(-5, 100)
plt.show()

In [ ]:
X_test = sm.add_constant(X_test)

In [ ]:
y_test_pred=model_trained.predict(X_test.drop(columns=['10cm 지중온도(°C)', '5cm 지중온도(°C)', '일산화탄소', '이슬점온도(°C)']))
y_test_pred

In [ ]:
# Mean Squrd Error
print(f'MSE : {mean_squared_error(y_train, y_train_pred)}')

# Root Mean Squrd Error
print(f'RMSE : {np.sqrt(mean_squared_error(y_train, y_train_pred))}')

# Root Mean Absolute Error
print(f'MAE : {mean_absolute_error(y_train, y_train_pred)}')

def Mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs(y_true - y_pred) / y_true) * 100

# Mean Absolute Percentage Error
print(f'MAPE : {Mean_absolute_percentage_error(y_train, y_train_pred)}') # Mean_absolute_percentage_error

print(f'R2 : {r2_score(y_train, y_train_pred)}') # r2_score

In [ ]:
# Mean Squrd Error
print(f'MSE : {mean_squared_error(y_test, y_test_pred)}')

# Root Mean Squrd Error
print(f'RMSE : {np.sqrt(mean_squared_error(y_test, y_test_pred))}')

# Root Mean Absolute Error
print(f'MAE : {mean_absolute_error(y_test, y_test_pred)}')

def Mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs(y_true - y_pred) / y_true) * 100

# Mean Absolute Percentage Error
print(f'MAPE : {Mean_absolute_percentage_error(y_test, y_test_pred)}') # Mean_absolute_percentage_error

print(f'R2 : {r2_score(y_test, y_test_pred)}') # r2_score

# 시계열처리 - Tensorflow

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
tf.random.set_seed(777) #하이퍼파라미터 튜닝을 위해 실행시 마다 변수가 같은 초기값 가지게 하기
import numpy as np
#matplotlib 패키지 한글 깨짐 처리 시작
import matplotlib.pyplot as plt
import platform
if platform.system() == 'Darwin': #맥
        plt.rc('font', family='AppleGothic') 
elif platform.system() == 'Windows': #윈도우
        plt.rc('font', family='Malgun Gothic') 
elif platform.system() == 'Linux': #리눅스 (구글 콜랩)
        #!wget "https://www.wfonts.com/download/data/2016/06/13/malgun-gothic/malgun.ttf"
        #!mv malgun.ttf /usr/share/fonts/truetype/
        #import matplotlib.font_manager as fm 
        #fm._rebuild() 
        plt.rc('font', family='Malgun Gothic') 
plt.rcParams['axes.unicode_minus'] = False #한글 폰트 사용시 마이너스 폰트 깨짐 해결
#matplotlib 패키지 한글 깨짐 처리 끝
%matplotlib inline

In [ ]:
def load_time_series_data(data, sequence_length):
    #print(data.shape) #(1225, 1)
    #print(sequence_length) #3
    window_length = sequence_length + 1
    x_data = []
    y_data = []
    for i in range(0, len(data) - window_length + 1): #0 1 2 3 4 5 6 7 8 9 | 10
        window = data[i:i + window_length, :]
        x_data.append(window[:-1, :])
        y_data.append(window[-1, [-1]])
    x_data = np.array(x_data)
    y_data = np.array(y_data)
    #print(x_data.shape) #(1222, 3, 1)
    #print(y_data.shape) #(1222, 1)

    return x_data, y_data

In [ ]:
df_input.columns

In [ ]:
data = df_input[['6h_PM2.5']].to_numpy()
print(data.shape) #((720, 1)

In [ ]:
transformer = MinMaxScaler()
data = transformer.fit_transform(data)

In [ ]:
df_input.shape

In [ ]:
sequence_length = 26
x_data, y_data = load_time_series_data(data, sequence_length)
x_data = x_data.reshape(len(x_data), -1)
print(x_data.shape) #(69518, 3)
print(y_data.shape) #(69518, 1)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, shuffle=False) #시각화를 위해 shuffle=False 옵션 사용
print(x_train.shape)  # (48662, 3)
print(y_train.shape) # (48662, 1)
print(x_test.shape)# (20856, 3)
print(y_test.shape) # (20856, 1)

In [ ]:
##########모델 생성

input = tf.keras.layers.Input(shape=(sequence_length,))
net = tf.keras.layers.Dense(units=32, activation='relu')(input)
net = tf.keras.layers.Dense(units=32, activation='relu')(net)
net = tf.keras.layers.Dense(units=1)(net)
model = tf.keras.models.Model(input, net)

##########모델 학습

model.compile(loss='mean_squared_error', optimizer='adam')

model.fit(x_train, y_train, epochs=40, validation_data=(x_test, y_test)) 

##########모델 검증

In [ ]:
plt.figure(figsize=(20, 20))
plt.plot(df_input['6h_PM2.5'])

In [ ]:
##########모델 예측                           

def plot(data, y_predict_train, y_predict_test):
#     plt.figure(figsize=(20, 20))
    plt.plot(transformer.inverse_transform(data)[:, [-1]].flatten(), label='실제 종가')

    y_predict_train = transformer.inverse_transform(y_predict_train)
    y_predict_train_plot = np.empty_like(data[:, [0]])
    y_predict_train_plot[:, :] = np.nan
    y_predict_train_plot[sequence_length:len(y_predict_train) + sequence_length, :] = y_predict_train
    plt.plot(y_predict_train_plot.flatten(), label='학습 데이터 예측 종가')

    y_predict_test = transformer.inverse_transform(y_predict_test)
    y_predict_test_plot = np.empty_like(data[:, [0]])
    y_predict_test_plot[:, :] = np.nan
    y_predict_test_plot[len(y_predict_train) + sequence_length:, :] = y_predict_test
    plt.plot(y_predict_test_plot.flatten(), label='테스트 데이터 예측 종가')

    plt.legend()
    plt.show()

y_predict_train = model.predict(x_train)
y_predict_test = model.predict(x_test)
plot(data, y_predict_train, y_predict_test)

# # 검증
# x_test = np.array([
#         [44, 49, 50]
# ])
# x_test = x_test.reshape(-1, 1)
# x_test = transformer.transform(x_test)
# x_test = x_test.reshape(1, sequence_length)

# y_predict = model.predict(x_test)

# y_predict = transformer.inverse_transform(y_predict)
# print(y_predict[0][0]) #51.707638

In [ ]:
# rnn, LSTM

In [ ]:
df_RNN = df_input[['PM10', 'PM2.5', '오존', '이산화질소', '일산화탄소', '아황산가스', '기온(°C)',
       '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)', '증기압(hPa)', '이슬점온도(°C)',
       '현지기압(hPa)', '해면기압(hPa)', '지면온도(°C)', '5cm 지중온도(°C)', '10cm 지중온도(°C)',
       '20cm 지중온도(°C)', '30cm 지중온도(°C)', 'year', 'month', 'day', 'hour',
       '6h_PM2.5']]

In [ ]:
sequence_x = 7*24
sequence_y = 24
m = len(df_RNN) - (sequence_x + sequence_y) +1
m

In [ ]:
df_RNN.shape

In [ ]:
feature_k = df_RNN.shape[-1]
(m, sequence_x, feature_k),(m, sequence_y, 1) 

In [ ]:
data_x = np.zeros((m, sequence_x, feature_k), np.float32)
data_y = np.zeros((m, sequence_y, 1), np.float32)

In [ ]:
np.array(df_RNN).shape

In [ ]:
df_arr = np.array(df_RNN)
df_arr.shape


In [ ]:
df_RNN.head()

In [ ]:
df_arr_i = df_arr.tolist()

In [ ]:
fg = pd.DataFrame(df_arr_i[:5], columns= df_arr_i)
fg

In [ ]:
for i in range(m):
    data_x[i] = df_arr[i : i+sequence_x]
    data_y[i] = df_arr[i + sequence_x : i + sequence_x + sequence_y + -1]

**참고 : https://github.com/rickiepark/deep-learning-with-python-notebooks/blob/master/6.4-sequence-processing-with-convnets.ipynb**



In [ ]:
from keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence



max_features = 10000  # 특성으로 사용할 단어의 수
max_len = 500  # 사용할 텍스트의 길이(가장 빈번한 max_features 개의 단어만 사용합니다)

print('데이터 로드...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), '훈련 시퀀스')
print(len(x_test), '테스트 시퀀스')

print('시퀀스 패딩 (samples x time)')
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)
print('x_train 크기:', x_train.shape)
print('x_test 크기:', x_test.shape)

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

model = Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))

model.summary()

model.compile(optimizer=RMSprop(lr=1e-4),
              loss='mean_squared_error',
              metrics=['loss'])
history = model.fit(x_train, y_train,
                    epochs=50,
                    batch_size=128,
                    validation_data=(x_test, y_test))
